# Vehicle routing with Timefold in a Kotlin notebook

In [32]:
@file:DependsOn("ai.timefold.solver:timefold-solver-core:1.7.0")
@file:DependsOn("com.fasterxml.jackson.module:jackson-module-kotlin:2.16.1")

## Domain

Create the domain data classes:

### Location

In [33]:
import com.fasterxml.jackson.annotation.JsonFormat

@JsonFormat(shape = JsonFormat.Shape.ARRAY)
data class Location(
    val latitude: Double,
    val longitude: Double) {
    
    fun calcEuclideanDistanceTo(other: Location): Double {
        val xDifference = latitude - other.latitude
        val yDifference = longitude - other.longitude
        return Math.sqrt(xDifference * xDifference + yDifference * yDifference)
    }
    
}

### Visit

In [34]:
import java.time.Duration

data class Visit(

    val name: String,
    val location: Location,
    val load: Int) {

    override fun toString(): String = name
}

### Vehicle

In [35]:
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.variable.PlanningListVariable


@PlanningEntity
data class Vehicle(

    val name: String,
    val homeLocation: Location,
    val capacity: Int) {

    @PlanningListVariable
    var visits: MutableList<Visit> = ArrayList()
    
    // No-arg constructor required for Timefold
    constructor() : this("", Location(0.0, 0.0), 0)

    override fun toString(): String = name
}

## Constraints

In [36]:
import ai.timefold.solver.core.api.score.buildin.hardsoftlong.HardSoftLongScore
import ai.timefold.solver.core.api.score.stream.Constraint
import ai.timefold.solver.core.api.score.stream.ConstraintFactory
import ai.timefold.solver.core.api.score.stream.ConstraintProvider
import ai.timefold.solver.core.api.score.stream.Joiners
import java.time.Duration

class VehicleRoutingConstraintProvider : ConstraintProvider {

    override fun defineConstraints(constraintFactory: ConstraintFactory): Array<Constraint>? {
        return arrayOf(
            // Hard constraints
            capcity(constraintFactory),
            // Soft constraints
            minimizeDistance(constraintFactory)
        )
    }
    
    fun capcity(constraintFactory: ConstraintFactory): Constraint {
        // TODO Not the most efficient implementation
        return constraintFactory
            .forEach(Vehicle::class.java)
            .expand({ vehicle -> vehicle.visits.sumOf { it.load } })
            .filter({ vehicle, load -> load > vehicle.capacity })
            .penalizeLong(HardSoftLongScore.ONE_HARD,
                { vehicle, load -> (load - vehicle.capacity).toLong() })
            .asConstraint("vehicle-routing", "Capacity");
    }
    
    fun minimizeDistance(constraintFactory: ConstraintFactory): Constraint {
        // TODO Not the most efficient implementation
        return constraintFactory
            .forEach(Vehicle::class.java)
            .penalizeLong(HardSoftLongScore.ONE_SOFT, { vehicle ->
                var distance: Double = 0.0
                var previousLocation: Location = vehicle.homeLocation
                for (visit in vehicle.visits) {
                    distance += previousLocation.calcEuclideanDistanceTo(visit.location)
                    previousLocation = visit.location
                }
                distance += previousLocation.calcEuclideanDistanceTo(vehicle.homeLocation)
                (distance * 1_000_000.0).toLong()
            })
            .asConstraint("vehicle-routing", "Minimize distance");
    }

}

## Schedule

In [37]:
import ai.timefold.solver.core.api.domain.solution.PlanningEntityCollectionProperty
import ai.timefold.solver.core.api.domain.solution.PlanningScore
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.solution.ProblemFactCollectionProperty
import ai.timefold.solver.core.api.domain.valuerange.ValueRangeProvider
import ai.timefold.solver.core.api.score.buildin.hardsoftlong.HardSoftLongScore
import kotlinx.serialization.Transient


@PlanningSolution
data class Schedule(
    
    val name: String,
    @PlanningEntityCollectionProperty
    val vehicles: List<Vehicle>,
    @ProblemFactCollectionProperty
    @ValueRangeProvider
    val visits: List<Visit>) {

    @PlanningScore
    @Transient
    var score: HardSoftLongScore? = null

    // No-arg constructor required for Timefold
    constructor() : this("", emptyList(), emptyList())

}

## Read input data

In [38]:
import java.io.File
import com.fasterxml.jackson.module.kotlin.jacksonObjectMapper
import com.fasterxml.jackson.module.kotlin.readValue

val mapper = jacksonObjectMapper()
val problem: Schedule =  mapper.readValue(File("vehicle-routing-data.json"))

## Solve

In [39]:
import ai.timefold.solver.core.config.solver.SolverConfig
import ai.timefold.solver.core.api.solver.SolverFactory
import ai.timefold.solver.core.api.solver.Solver

val solverFactory: SolverFactory<Schedule> = SolverFactory.create(SolverConfig()
        .withSolutionClass(Schedule::class.java)
        .withEntityClasses(Vehicle::class.java)
        .withConstraintProviderClass(VehicleRoutingConstraintProvider::class.java)
        // The solver runs only for 5 seconds on this small dataset.
        // It's recommended to run for at least 5 minutes ("5m") otherwise.
        .withTerminationSpentLimit(Duration.ofSeconds(30)))

println("Solving the problem ...")
val solver: Solver<Schedule> = solverFactory.buildSolver()
val solution: Schedule = solver.solve(problem)
println("Solving finished with score (${solution.score}).")

Solving the problem ...
Solving finished with score (0hard/-24014318soft).


## Visualize the schedule

In [40]:
HTML(buildString {
    append("<p style='font-size: x-large'>Score: ${solution.score}</p>")
    append("<ul>")
    for (vehicle in solution.vehicles) {
        append("<li>${vehicle.name}: ${vehicle.visits.joinToString(", ")}</li>")
    }
    append("</ul>")
})

Score: 0hard/-24014318soft Vehicle A: FONTAINE-L'EVEQUE, HEPPIGNIES, SAINT-MARTIN Vehicle B: OLLIGNIES, MAULDE, GUIGNIES, BAVIKHOVE, WETTEREN Vehicle C: SINT-KWINTENS-LENNIK, NAAST, BERSILLIES-L'ABBAYE, BOURLERS, DONSTIENNES, LES_BONS_VILLERS Vehicle D: JURBISE, CHIEVRES, MAFFLE, GONDREGNIES, PEPINGEN Vehicle E: VOSSEM, WARNANT-DREYE, EVELETTE, MONT_NAM., ORET, SOMZEE Vehicle F: ANTHISNES, TAILLES, SELANGE, VAUX-LEZ-ROSIERES, POUPEHAN, AVE-ET-AUFFE Vehicle G: HUMBEEK, HOFSTADE_BT., HAREN_BRUSSEL Vehicle H: MERKSPLAS, ROSMEER, VILLERS-L'EVEQUE, GELINDEN Vehicle I: TILFF, XHENDELESSE, BLERET Vehicle J: KOMEN, ELVERDINGE, RENINGE, BULSKAMP, LISSEWEGE, DAMME, LANDEGEM, NIEUWKERKEN-WAAS

## Analyze the score

Let's break down the score per constraint:

In [41]:
import ai.timefold.solver.core.api.solver.SolutionManager

val solutionManager = SolutionManager.create(solverFactory)
val scoreAnalysis = solutionManager.analyze(solution)

And visualize it:

In [42]:
HTML(buildString {
    append("<p style='font-size: x-large'>Score: ${scoreAnalysis.score}</p>")
    append("<ul>")
    for (constraint in scoreAnalysis.constraintMap().values) {
        append("<li>${constraint.constraintRef().constraintName}: ${constraint.score.toShortString()}</li>")
    }
    append("</ul>")
})

Score: 0hard/-24014318soft Minimize distance: -24014318soft Capacity: 0

## Visualization

Let's visualize the solution:

In [43]:
%use lets-plot
%use lets-plot-gt

### Map

In [44]:
val locations = mutableListOf<Location>()
solution.vehicles.forEach { vehicle ->
    locations.add(vehicle.homeLocation)
    locations.addAll(vehicle.visits.map { it.location })
    locations.add(vehicle.homeLocation)
}

val dataset = mapOf(
        "latitude" to locations.map { it.latitude },
        "longitude" to locations.map { it.longitude },
)

letsPlot(dataset) + geomPath() { x = "longitude"; y = "latitude" }

## Statistics

For a big dataset, a schedule visualization is often too verbose.
Let's visualize the solution through statistics:

In [45]:
%use kandy

### Visits per vehicle

In [46]:
val vehicles = solution.vehicles.map { it.name }
val visitCounts = solution.vehicles.map { it.visits.size }

plot {
    layout.title = "Visits per vehicle"
    bars {
        x(vehicles) { axis.name = "Vehicle" }
        y(visitCounts) { axis.name = "Visits" }
    }
}

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="WeWKZ7"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Visits per vehicle"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Vehicle"
},{
"aesthetic":"y",
"name":"Visits",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":["Vehicle A","Vehicle B","Vehicle C","Vehicle D","Vehicle E","Vehicle F","Vehicle G","Vehicle H","Vehicle I","Vehicle J"],
"y":[3.0,5.0,6.0,5.0,6.0,6.0,3.0,4.0,3.0,8.0]
},
"sampling":"none",
"position":"dodge",
"geom":"bar"
}]
};
 var plotContainer = document.getElementById("WeWKZ7");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Vehicle A
 
 
 
 
 
 
 
 
 Vehicle B
 
 
 
 
 
 
 
 
 Vehicle C
 
 
 
 
 
 
 
 
 Vehicle D
 
 
 
 
 
 
 
 
 Vehicle E
 
 
 
 
 
 
 
 
 Vehicle F
 
 
 
 
 
 
 
 
 Vehicle G
 
 
 
 
 
 
 
 
 Vehicle H
 
 
 
 
 
 
 
 
 Vehicle I
 
 
 
 
 
 
 
 
 Vehicle J
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 2
 
 
 
 
 
 
 4
 
 
 
 
 
 
 6
 
 
 
 
 
 
 8
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Visits per vehicle
 
 
 
 
 Visits
 
 
 
 
 Vehicle

### Load per vehicle

In [47]:
val vehicles = solution.vehicles.map { it.name }
val load = solution.vehicles.map { it.visits.sumOf { it.load } }

plot {
    layout.title = "Load per vehicle"
    bars {
        x(vehicles) { axis.name = "Vehicle" }
        y(load) { axis.name = "Load" }
    }
}

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="FJ1gMN"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Load per vehicle"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Vehicle"
},{
"aesthetic":"y",
"name":"Load",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":["Vehicle A","Vehicle B","Vehicle C","Vehicle D","Vehicle E","Vehicle F","Vehicle G","Vehicle H","Vehicle I","Vehicle J"],
"y":[89.0,93.0,98.0,89.0,99.0,100.0,76.0,94.0,98.0,98.0]
},
"sampling":"none",
"position":"dodge",
"geom":"bar"
}]
};
 var plotContainer = document.getElementById("FJ1gMN");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Vehicle A
 
 
 
 
 
 
 
 
 Vehicle B
 
 
 
 
 
 
 
 
 Vehicle C
 
 
 
 
 
 
 
 
 Vehicle D
 
 
 
 
 
 
 
 
 Vehicle E
 
 
 
 
 
 
 
 
 Vehicle F
 
 
 
 
 
 
 
 
 Vehicle G
 
 
 
 
 
 
 
 
 Vehicle H
 
 
 
 
 
 
 
 
 Vehicle I
 
 
 
 
 
 
 
 
 Vehicle J
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 20
 
 
 
 
 
 
 40
 
 
 
 
 
 
 60
 
 
 
 
 
 
 80
 
 
 
 
 
 
 100
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Load per vehicle
 
 
 
 
 Load
 
 
 
 
 Vehicle

## Conclusion

To learn more about planning optimization, visit [timefold.ai](https://timefold.ai).